In [26]:
# Necessary imports
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [32]:
df_pos = pd.read_csv(open('clean_positive_train.csv','r'), encoding='utf-8', engine='c')
df_neg = pd.read_csv(open('clean_negative_train.csv','r'), encoding='utf-8', engine='c')

df_pos['text'] = df_pos['text'].astype(str)
df_pos['parent_text'] = df_pos['parent_text'].astype(str)

df_neg['text'] = df_neg['text'].astype(str)
df_neg['parent_text'] = df_neg['parent_text'].astype(str)

In [3]:
# df_pos.describe()

,score,ups,controversiality,parent_score,parent_ups,parent_controversiality
count,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000
mean,198.448509,198.448509,0.000020,369.154003,369.154003,0.000700
std,256.498200,256.498200,0.004472,530.071252,530.071252,0.026449
min,66.000000,66.000000,0.000000,-8907.000000,-8907.000000,0.000000
25%,83.000000,83.000000,0.000000,84.000000,84.000000,0.000000
50%,116.000000,116.000000,0.000000,185.000000,185.000000,0.000000
75%,201.000000,201.000000,0.000000,419.000000,419.000000,0.000000
max,4865.000000,4865.000000,1.000000,9531.000000,9531.000000,1.000000


In [4]:
# df_neg.describe()

,score,ups,controversiality,parent_score,parent_ups,parent_controversiality
count,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000
mean,-14.564351,-14.564351,0.001460,66.805736,66.805736,0.002740
std,15.274812,15.274812,0.038183,216.581912,216.581912,0.052274
min,-634.000000,-634.000000,0.000000,-1622.000000,-1622.000000,0.000000
25%,-15.000000,-15.000000,0.000000,6.000000,6.000000,0.000000
50%,-10.000000,-10.000000,0.000000,15.000000,15.000000,0.000000
75%,-8.000000,-8.000000,0.000000,44.000000,44.000000,0.000000
max,-6.000000,-6.000000,1.000000,14776.000000,14776.000000,1.000000


Now that we have a small part of clean text we can start to learn the texts to our machine

In [34]:
# First we concatenate both lists and shuffle it to scrabble positive and negative
df = pd.concat([df_pos, df_neg])
df = df.sample(frac=1).reset_index(drop=True)
df.dropna(axis=0, inplace=True)

# Second we split our data for training and testing
text_data = df['text']
text_score = df['score']
# parent_text_data = df['parent_text']
# parent_text_score = df['parent_score']
X_train, X_test, y_train, y_test = train_test_split(text_data,text_score, test_size = 0.20, random_state = 42)

print(min(y_train))
print(max(y_train))
print(y_train.shape)


def posneg(number):
    if number > 0:
        return 1
    else:
        return 0
    
y_train = y_train.apply(posneg)
y_test = y_test.apply(posneg)

-634
4865
(79998,)


In [35]:
# We create a tokenizer which will give a word_index integer value to each word
tokenizer = Tokenizer(num_words=10000, lower=True, split=' ', document_count=0)

# Create the word_index list based on all our data
tokenizer.fit_on_texts(text_data)

# Now we make a list of sequences of integers based on our texts
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [36]:
train_data = keras.preprocessing.sequence.pad_sequences(X_train_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=128)

test_data = keras.preprocessing.sequence.pad_sequences(X_test_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=128)

In [8]:
print(len(train_data[0]), len(train_data[1]))
print(max(train_data, key=len))
print(max(test_data, key=len))
print(len(tokenizer.word_index) + 1)

128 128
[7462  210  368 5123   21  287   44  176    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[ 122   26  192   90  752 9054  154 1367   25  262  243 1437    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0  

## Finally we can do some neural networks!

In [9]:
vocab_size = len(tokenizer.word_index) + 1


model = tf.keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(24, activation=tf.nn.relu))
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          1468352   
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                400       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 1,469,449
Trainable params: 1,469,449
Non-trainable para

In [10]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [37]:
history = model.fit(train_data,
                    y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(test_data, y_test),
                    verbose=1)

Train on 79998 samples, validate on 20000 samples
Epoch 1/40
79998/79998 [==============================] - 2s 27us/sample - loss: 0.7528 - acc: 0.5841 - val_loss: 0.6813 - val_acc: 0.5913
Epoch 2/40
79998/79998 [==============================] - 2s 27us/sample - loss: 0.6641 - acc: 0.6109 - val_loss: 0.6678 - val_acc: 0.6018
Epoch 3/40
79998/79998 [==============================] - 2s 26us/sample - loss: 0.6485 - acc: 0.6277 - val_loss: 0.6586 - val_acc: 0.6189
Epoch 4/40
79998/79998 [==============================] - 2s 26us/sample - loss: 0.6358 - acc: 0.6418 - val_loss: 0.6541 - val_acc: 0.6206
Epoch 5/40
79998/79998 [==============================] - 2s 27us/sample - loss: 0.6245 - acc: 0.6520 - val_loss: 0.6471 - val_acc: 0.6327
Epoch 6/40
79998/79998 [==============================] - 2s 26us/sample - loss: 0.6128 - acc: 0.6636 - val_loss: 0.6413 - val_acc: 0.6425
Epoch 7/40
79998/79998 [==============================] - 2s 26us/sample - loss: 0.6015 - acc: 0.6753 - val_loss: 0.